In [ ]:
# import libraries
import io
import os
import sys
import uuid
import pickle

import google.generativeai as genai
import gradio as gr
import numpy as np

from PIL import Image
from dotenv import load_dotenv

from tensorflow import keras
from tensorflow.keras.preprocessing import image


sys.path.append(os.path.abspath(r"./utilities"))
import MovieScriptGenerator
import MoviePosterGenerator

In [2]:
# temporary directory to save images that the user uploads
SAVE_DIR = "temp_images"
os.makedirs(SAVE_DIR, exist_ok=True)

In [3]:
def run_prediction(filePath):
    print("Running prediction")
    model = keras.models.load_model("../models/keras_model.h5")

    img = Image.open(filePath)
    img = img.resize((150, 150))

    img_array = image.img_to_array(img)
    img_array = img_array / 255.0 
    img_array = np.expand_dims(img_array, axis=0)

    shape = img_array.shape
    dtype = (img_array.dtype)

    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions)
    # Print predictions
    print(predictions)
    print(predicted_class)
    return f"""Shape = {shape}
    Predictions = {predictions}
    Predicted class = {predicted_class}"""

    

In [4]:
def generate(image):
    """
    Processes an input image to generate a movie script and a resized movie poster.

    Args:
        image (PIL.Image.Image): The input image.

    Returns:
        tuple: (str, PIL.Image.Image)
            - The generated movie script as a string.
            - The resized movie poster as a PIL.Image.Image object.
    """
    file_path = os.path.join(SAVE_DIR, f"{uuid.uuid4().hex}.jpg")
    image.save(file_path)
    movieScriptGenerator = MovieScriptGenerator.MovieScriptGenerator()
    script = movieScriptGenerator.generate_script(file_path)
    moviePosterGenerator = MoviePosterGenerator.MoviePosterGenerator()
    poster = moviePosterGenerator.generate_poster(script)
    new_size = (300, 300)
    resized_poster = poster.resize(new_size, Image.LANCZOS)
    predict = run_prediction(file_path)
    return predict, script, resized_poster

In [ ]:
# Create a Gradio interface for generating a movie script and poster from an image

app = gr.Interface(
    fn=generate,
    inputs=gr.Image(type="pil"), 
    outputs=[gr.Textbox(label="Predict Information"),
             gr.Textbox(label="Movie information"), 
             gr.Image(type="pil", label="Movie Poster")],  
    title="Movie Script Generator and Poster Generator",
    description="Paste or upload an image. The system will generate movie information and create a poster.")
# Launch the app
app.launch(share=True)

In [ ]:
# Version 2: Gradio UI with Blocks for better layout

with gr.Blocks() as demo:
    # 🎬 Title
    gr.Markdown("## 🎥 Movie Script & Poster Generator 🎭")
    
    with gr.Row():
        with gr.Column():
            gr.Markdown("### Upload an Image 🎨")
            image_input = gr.Image(type="pil", label="Paste or Upload Image")  
            generate_button = gr.Button("🎬 Generate Movie")  # Added button for better UX

    with gr.Row():
        with gr.Column():
            gr.Markdown("### Predicted Movie Information 📜")
            text_predict = gr.Textbox(label="Prediction Information", interactive=False)
    
    with gr.Row():
        with gr.Column():
            gr.Markdown("### 📜 Generated Movie Script")
            text_output = gr.Textbox(label="Movie Information", interactive=False)
        
        with gr.Column():
            gr.Markdown("### 🎨 Generated Movie Poster")
            image_output = gr.Image(type="pil", label="Movie Poster")

    generate_button.click(fn=generate, inputs=image_input, outputs=[text_predict, text_output, image_output])

# Launch the UI
demo.launch(share=True)